In [3]:
#Loads packages
import numpy as np
import pandas as pd
import tifffile
import napari
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from skimage.filters import threshold_multiotsu, threshold_otsu
from skimage.morphology import ball, binary_dilation,erosion
from skimage.exposure import rescale_intensity
from skimage.measure import label, regionprops

In [ ]:
#Function to remove segments smaller than min_size voxels
def erase_small(labels, min_size):
    unique, counts = np.unique(labels, return_counts=True)
    for i in range(len(unique)):
        if counts[i]<min_size:
            labels[labels==unique[i]]=0
    return labels

In [ ]:
#Function to remove background from imaging data
def remove_backg(img, sig1,sig2):
    #img = img.astype('int16')
    img_sig =ndi.gaussian_filter(img, (sig1, sig1,sig1))
    img_bcg = ndi.gaussian_filter(img, (sig2,sig2,sig2))
    cleaned = img_sig - img_bcg
    cleaned[cleaned<0]=0
    return cleaned

In [ ]:
#Function to create a DataFrame from instance segmentation
def make_obj_table(labels):
    props = regionprops(labels)
    Results = pd.DataFrame(columns=['id', 'area'])
    ids=[]
    areas=[]
    [(ids.append(r.label),areas.append(r.area)) for r in props]
    Results['id']=ids
    Results['area']=areas
    return Results

In [ ]:
#Function to compute IOU for one pair of objects
def per_object_IoU(labels_immuno, labels_predicted, immuno_id, predicted_id):
    mask_immuno = labels_immuno == immuno_id
    mask_predicted = labels_predicted == predicted_id

    intersection = np.logical_and(mask_immuno, mask_predicted)
    union = np.logical_or(mask_immuno, mask_predicted)

    iou = np.sum(intersection)/np.sum(union)
    return iou

In [ ]:
#Function to compute Dice coefficient for one pair of objects
def per_object_Dice(labels_immuno, labels_predicted, immuno_id, predicted_id):
    mask_immuno = labels_immuno == immuno_id
    mask_predicted = labels_predicted == predicted_id

    intersection = np.logical_and(mask_immuno, mask_predicted)
    #union = np.logical_or(mask_immuno, mask_predicted)

    dice = 2*np.sum(intersection)/(np.sum(mask_immuno) + np.sum(mask_predicted))
    return dice

This notebook is used to validate the performance of deep-learning-based pSCR segmentation. The algorithm takes coCATS (N2V), immunostained and predicted BASSOON imaging data as input. 

Two validation metrics are computed: voxel-based (Pearson correlation), and object-based (F1 curve)


#### Load imaging data

In [ ]:
#Loads imaging data and BASSOON prediction
img_path_cats = '.\\data\\cats.tiff'
img_path_bassoon_immuno = '.\\data\\immuno_bassoon.tiff'
img_path_bassoon_predicted = '.\\data\\predicted_bassoon.tiff'

cats = tifffile.imread(img_path_cats)[0,:,:,:]
bassoon_immuno = tifffile.imread(img_path_bassoon_immuno)[0,:,:,:]
bassoon_predicted = tifffile.imread(img_path_bassoon_predicted)[0,:,:,:]

#### Metric 1: Pearson correlation between immuno and predicted BASSOON

In [ ]:
#Computes voxel-wise Pearson correlation coefficient between immuno and predicted BASSOON
array_a = np.ndarray.flatten(bassoon_immuno)
array_b = np.ndarray.flatten(bassoon_predicted)
correlation = np.corrcoef(array_a, array_b)
correlation[1,0]

#### Metric 2: Object-based validation

In [ ]:
#Creates masks of immuno and predicted BASSOON data
threshhold_bassoon_immuno = threshold_otsu(bassoon_immuno)
mask_bassoon_immuno = bassoon_immuno > threshhold_bassoon_immuno
threshhold_bassoon_predicted = threshold_otsu(bassoon_predicted)
mask_bassoon_predicted = bassoon_predicted > threshhold_bassoon_predicted

In [ ]:
#Stretches contrast of coCATS data
vmin, vmax = np.quantile(cats, q=(0.01, 0.99))
stretched_cats = rescale_intensity(
    cats, 
    in_range=(vmin, vmax), 
)
#remove background from coCATS data
cats_clean = remove_backg(stretched_cats, 0.3,7)

#perform errosion operation on coCATS data
cats_eroded = erosion(cats_clean, ball(1))

In [ ]:
#Thresholds erroded coCATS data
thresholds = threshold_multiotsu(cats_eroded)
cats_mask = cats_eroded>thresholds[1]

In [ ]:
#Cropps coCATS data according to dilated immuno BASSOON mask
mask_cats_immuno = cats_mask*binary_dilation(mask_bassoon_immuno, ball(1))
#Creates pSCRimmuno instance segmentation
labels_immuno = label(mask_cats_immuno)
labels_immuno  = erase_small(labels_immuno, 6)

In [ ]:
#Cropps coCATS data according to dilated predicted BASSOON mask
mask_cats_predicted = cats_mask*binary_dilation(mask_bassoon_predicted, ball(1))
#Creates pSCRpredicted instance segmentation
labels_predicted = label(mask_cats_predicted)
labels_predicted = erase_small(labels_predicted, 6)

In [ ]:
#Sets up napari Viewer and displays the results
viewer = napari.Viewer()
viewer.add_image(cats,colormap='gray_r', name='CATS')
viewer.add_image(bassoon_immuno, name='bassoon immuno', colormap = 'magenta')
viewer.add_image(bassoon_predicted, name='bassoon predicted', colormap = 'magenta')
viewer.add_labels(labels_immuno,name='pSCRs immuno')
viewer.add_labels(labels_predicted,name='pSCRs predicted')

In [ ]:
#Finds matches between pSCRimmuno and pSCRpredicted segments
rp = regionprops(labels_immuno, intensity_image=labels_predicted)
selection = [x for x in rp if x.max_intensity > 0]
tab1= make_obj_table(labels_immuno)
tab1 = tab1.set_index('id')
tab1['match_list']=''
tab1['ious']=''
tab1['max iou']=0
for i in selection:
    a,counts = np.unique(i.intensity_image,  return_counts=True)
    tab1['match_list'].loc[i.label ]= a[a!= 0]
    ious=[]
    matches = a[a!= 0]
    for j in range(len(matches)):
        immuno_id = i.label
        predicted_id = matches[j]
        ious.append(per_object_IoU(labels_immuno, labels_predicted, immuno_id, predicted_id))
    tab1['ious'].loc[i.label ]=ious
    tab1['max iou'].loc[i.label]=max(ious)

In [ ]:
#Finds matches between pSCRpredicted and pSCRimmuno segments
rp = regionprops(labels_predicted, intensity_image=labels_immuno)
selection = [x for x in rp if x.max_intensity > 0]
tab2= make_obj_table(labels_predicted)
tab2 = tab2.set_index('id')
tab2['match_list']=''
tab2['ious']=''
tab2['max iou']=0
for i in selection:
    a,counts = np.unique(i.intensity_image,  return_counts=True)
    tab2['match_list'].loc[i.label ]= a[a!= 0]
    ious=[]
    matches = a[a!= 0]
    for j in range(len(matches)):
        predicted_id = i.label
        immuno_id = matches[j]
        ious.append(per_object_IoU(labels_predicted, labels_immuno, predicted_id, immuno_id))
    tab2['ious'].loc[i.label ]=ious
    tab2['max iou'].loc[i.label]=max(ious)

In [ ]:
#Calculates validation metrics per IOU threshold
ious = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
recalls=[]
precisions=[]
f1s=[]
for thr in ious:
    TP = len(tab1[tab1['max iou']>thr])
    FN = len(tab1)-TP
    TP2 = len(tab2[tab2['max iou']>thr])
    FP = len(tab2)-TP2
    p=TP2/(TP2+FP)
    r = TP2/(TP2+FN)
    recalls.append(r)
    precisions.append(p)
    if p==0 or r == 0:
        f1s.append(0)  
    else:
        f1s.append(2*r*p/(r+p))

In [ ]:
#Displays F1 curve
fig = plt.figure(figsize=(4,4))
plt.plot(ious,f1s, c='k')
plt.title('F1 per IOU')
plt.xlabel("IOU")
plt.ylabel("F1")
plt.ylim(0,1)
plt.xlim(0,0.9)